<a href="https://colab.research.google.com/github/rushikeshw791/Naive-Bayes-1/blob/main/NaiveBayesOnSalaryData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Naive Bayes**

1) Prepare a classification model using Naive Bayes 
for salary data 

Data Description:

age -- age of a person

workclass	-- A work class is a grouping of work 

education	-- Education of an individuals	

maritalstatus -- Marital status of an individulas	

occupation	 -- occupation of an individuals

relationship -- 	

race --  Race of an Individual

sex --  Gender of an Individual

capitalgain --  profit received from the sale of an investment	

capitalloss	-- A decrease in the value of a capital asset

hoursperweek -- number of hours work per week	

native -- Native of an individual

Salary -- salary of an individual

In [ ]:
# import librabraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import warnings
warnings.filterwarnings('ignore')

from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statsmodels.api as sm

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

**Salary train data**

In [ ]:
salary_train = pd.read_csv('https://github.com/rushikeshw791/Naive-Bayes-1/files/9840412/SalaryData_Train.csv')

In [ ]:
salary_train.head()

**Salary test data**

In [ ]:
salary_test =pd.read_csv('https://github.com/rushikeshw791/Naive-Bayes-1/files/9840414/SalaryData_Test.csv')

In [ ]:
salary_test.head()

**EDA on both data**

In [ ]:
salary_train.shape

(30161, 14)

In [ ]:
salary_train.head()

In [ ]:
salary_train.info()

In [ ]:
salary_train.describe()

Finding special characters in data 

In [ ]:
salary_train.isin(['?']).sum(axis=0)

In [ ]:
print(salary_train[0:5])

In [ ]:
# test data
salary_test.shape

In [ ]:
salary_test.head()

In [ ]:
salary_test.info()

In [ ]:
salary_test.describe()

In [ ]:
salary_test.isin(['?']).sum(axis=0)

In [ ]:
print(salary_test[0:5])

**For categorical variables**

In [ ]:
categorical = [var for var in salary_train.columns if salary_train[var].dtype=='O']

print('There are {} categorical variables\n'.format(len(categorical)))

print('The categorical variables are :\n\n',categorical)

In [ ]:
salary_train[categorical].head()

In [ ]:
# check missing values in categorical variables
salary_train[categorical].isnull().sum()

In [ ]:
# check frequency counts of values in categorical variables
 
for var in categorical:
  print(salary_train[var].value_counts())

In [ ]:
# view frequency distribution of categorical variables

for var in categorical:
  print(salary_train[var].value_counts()/np.float(len(salary_train)))

In [ ]:
# check labels in workclass variables
salary_train.workclass.unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' Self-emp-inc', ' Without-pay'], dtype=object)

In [ ]:
# check frequency distribution of values in workclass variables
salary_train.workclass.value_counts()

**For Occupation Variables**

In [ ]:
# check labels in occupation variables
salary_train.occupation.unique()

In [ ]:
# check freq distribution of values in occupation variables
salary_train.occupation.value_counts()

**For native country variable**

In [ ]:
# check labels in native-country variables
salary_train.native.unique()

In [ ]:
# check freq distribution of values in native-country variables
salary_train.native.value_counts()

**Cardinality - number of labels**

In [ ]:
# check for cardinality in categorical variables
for var in categorical:
  print(var,'contains',len(salary_train[var].unique()),'labels')

**For numerical variables**

In [ ]:
# find Numerical variables
numerical =[var for var in salary_train.columns if salary_train[var].dtype!='O']
print('There are {} numerical variables\n'.format(len(numerical)))
print('The numerical variables are :',numerical)

There are 5 numerical variables

The numerical variables are : ['age', 'educationno', 'capitalgain', 'capitalloss', 'hoursperweek']


In [ ]:
# view the numerical variables
salary_train[numerical].head()

In [ ]:
# check missing values in numerical variables
salary_train[numerical].isnull().sum()

**For feature vector and target variable**

In [ ]:
X = salary_train.drop(['Salary'],axis=1)
y = salary_train['Salary']

**Seperate data in training and testing set**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# split x and y into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)

In [ ]:
X_train.shape, X_test.shape

**Feature Engineering**

In [ ]:
X_train.dtypes

In [ ]:
X_test.dtypes

In [ ]:
# categorical variables
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']
categorical

In [ ]:
# numerical variables
numerical = [col for col in X_train.columns if X_train[col].dtypes!='O']
numerical

In [ ]:
# print percentage of missing values in the categorical variables in training set

X_train[categorical].isnull().mean()

In [ ]:
# print categorical variables with missing data

for col in categorical:
    if X_train[col].isnull().mean()>0:
        print(col, (X_train[col].isnull().mean()))

In [ ]:
# impute missing categorical variables with most frequent value

for df2 in [X_train, X_test]:
    df2['workclass'].fillna(X_train['workclass'].mode()[0], inplace=True)
    df2['occupation'].fillna(X_train['occupation'].mode()[0], inplace=True)
    df2['native'].fillna(X_train['native'].mode()[0], inplace=True) 

In [ ]:
# check missing values in categorical variables in X_train
X_train[categorical].isnull().sum()

In [ ]:
# check missing values in categorical variables in X_test
X_test[categorical].isnull().sum()

In [ ]:
# check missing values in X_train
X_train.isnull().sum()

In [ ]:
# check missing values in X_test
X_test.isnull().sum()

**Encoding Categorical variables**

In [ ]:
categorical

In [ ]:
X_train[categorical].head()

In [ ]:
!pip install category_encoders

In [ ]:
import category_encoders as ce

In [ ]:
# encode remaining variables with one-hot encoder
encoder = ce.OneHotEncoder(cols=['workclass', 'education', 'maritalstatus', 'occupation', 'relationship', 
                                 'race', 'sex', 'native'])
X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

In [ ]:
X_train.shape

In [ ]:
X_test.shape

**Feature Scaling**

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
cols=X_train.columns

In [ ]:
scalar = RobustScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [ ]:
X_train = pd.DataFrame(X_train, columns=[cols])

In [ ]:
X_test = pd.DataFrame(X_test, columns=[cols])

In [ ]:
X_train.head()

In [ ]:
X_test.head()

**Model Training**

Train a Guassian Naive Bayes classifier on the training set

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
# fix the model
gnb = GaussianNB()
gnb.fit(X_train, y_train)

**Predict the Results**

In [ ]:
y_pred = gnb.predict(X_test)
y_pred

**Accuracy Score**

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print('model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

**Compare the train-set and test-set accuracy**

In [ ]:
y_pred_train = gnb.predict(X_train)

y_pred_train

In [ ]:
print('Training-set accuracy score is {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

**Check for overfitting and underfitting**

In [ ]:
# print the scores on training and test set

print('Training set score is {:.4f}'.format(gnb.score(X_train, y_train)))

print('Test set score is {:.4f}'.format(gnb.score(X_test, y_test)))

**Compare model accuracy with null accuracy**

In [ ]:
# check class distribution in test set

y_test.value_counts()

In [ ]:
# check null accuracy score

null_accuracy = (7407/(7407+2362))

print('Null accuracy score is {0:0.4f}'. format(null_accuracy))

**Confusion matrix**

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# Print the Confusion Matrix and slice it into four pieces
cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
# visualize confusion matrix with heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

**Classification metrics**

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# Classification accuracy
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]

In [ ]:
# classification accuracy

classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)

print('Classification accuracy is {0:0.4f}'.format(classification_accuracy))

In [ ]:
# print classification error

classification_error = (FP + FN) / float(TP + TN + FP + FN)

print('Classification error is {0:0.4f}'.format(classification_error))

In [ ]:
# print precision score

precision = TP / float(TP + FP)


print('Precision Score is {0:0.4f}'.format(precision))

In [ ]:
recall = TP / float(TP + FN)

print('Recall or Sensitivity is {0:0.4f}'.format(recall))

In [ ]:
true_positive_rate = TP / float(TP + FN)


print('True Positive Rate is {0:0.4f}'.format(true_positive_rate))

In [ ]:
false_positive_rate = FP / float(FP + TN)


print('False Positive Rate is {0:0.4f}'.format(false_positive_rate))

In [ ]:
specificity = TN / (TN + FP)

print('Specificity score is {0:0.4f}'.format(specificity))

**Calculate class probability**

In [ ]:
# print the first 10 predicted probabilities of two classes- 0 and 1

y_pred_prob = gnb.predict_proba(X_test)[0:10]

y_pred_prob

In [ ]:
# store the probabilities in dataframe

y_pred_prob_df = pd.DataFrame(data=y_pred_prob, columns=['Prob of - <=50K', 'Prob of - >50K'])

y_pred_prob_df

In [ ]:
# print the first 10 predicted probabilities for class 1 - Probability of >50K

gnb.predict_proba(X_test)[0:10, 1]

In [ ]:
# store the predicted probabilities for class 1 - Probability of >50K

y_pred1 = gnb.predict_proba(X_test)[:, 1]

In [ ]:
# plot histogram of predicted probabilities

plt.rcParams['font.size'] = 12
plt.hist(y_pred1, bins = 10)
plt.title('Histogram of predicted probabilities of salaries >50K')
plt.xlim(0,1)
plt.xlabel('Predicted probabilities of salaries >50K')
plt.ylabel('Frequency')

**ROC Curve**

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
# plot ROC Curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred1, pos_label = '>50K')
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr, linewidth=2)
plt.plot([0,1], [0,1], 'k--' )
plt.rcParams['font.size'] = 12
plt.title('ROC curve for Gaussian Naive Bayes Classifier for Predicting Salaries')
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.show()

In [ ]:
# compute ROC AUC

from sklearn.metrics import roc_auc_score

ROC_AUC = roc_auc_score(y_test, y_pred1)

print('ROC AUC is {:.4f}'.format(ROC_AUC))

In [ ]:
# calculate cross-validated ROC AUC 

from sklearn.model_selection import cross_val_score

Cross_validated_ROC_AUC = cross_val_score(gnb, X_train, y_train, cv=5, scoring='roc_auc').mean()

print('Cross validated ROC AUC is {:.4f}'.format(Cross_validated_ROC_AUC))

**K - Fold Cross validation**

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
# Applying 10-Fold Cross Validation
scores = cross_val_score(gnb, X_train, y_train, cv = 10, scoring='accuracy')
print('Cross-validation scoresis {}'.format(scores))

In [ ]:
# compute Average cross-validation score

print('Average cross-validation score is {:.4f}'.format(scores.mean()))